In [1]:
import os

In [2]:
!mkdir /content/dataset/

In [3]:
!pip install datasets
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 kB 20.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 kB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 55.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 26.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 kB 29.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 kB 33.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 kB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 20.8 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 51.0 MB/s eta 0:00

In [4]:
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
import string
import re
import time

In [5]:
import json
import os
from pathlib import Path
from typing import *

import pandas as pd 
import scipy
from datasets import ClassLabel, Dataset
from sklearn import metrics
from sklearn.model_selection import train_test_split
import transformers
from transformers import (
    AutoModelForSequenceClassification,
    AutoTokenizer,
    DataCollatorWithPadding,
    EarlyStoppingCallback,
    Trainer,
    TrainingArguments,
)
print(transformers.__version__)

4.28.1


In [6]:
%cd /content/drive/MyDrive/CS5246Project/stock_dataset/
!cp Combined_train_3news.csv /content/dataset/
!cp Combined_valid_3news.csv /content/dataset/
!cp Combined_test_3news.csv /content/dataset/

df_train = pd.read_csv('/content/dataset/Combined_train_3news.csv')
df_valid = pd.read_csv('/content/dataset/Combined_valid_3news.csv')
df_test = pd.read_csv('/content/dataset/Combined_test_3news.csv')

/content/drive/MyDrive/CS5246Project/stock_dataset


In [ ]:

%cd /content/drive/MyDrive/5246Project/stock_dataset/
!cp Combined_News_DJIA.csv /content/dataset/

# Here we are interested in Combined_News_DJIA.csv file
df = pd.read_csv('/content/dataset/Combined_News_DJIA.csv')

In [8]:
df_train

,Date,Label,0,1,2,new_combined_news
0,2008-08-08,0,Georgia downs two Russian warplanes as countri...,Breaking Georgia invades South Ossetia Russia ...,Rice Gives Green Light for Israel to Attack Ir...,Georgia downs two Russian warplanes as countri...
1,2008-08-11,1,War in South Ossetia,Israel and the US behind the Georgian aggression,Jewish Georgian minister Thanks to Israeli tra...,War in South Ossetia Israel and the US behind ...
2,2008-08-12,0,Christopher King argues that the US and NATO a...,Remember that adorable year old who led your c...,Russia Georgia and NATO Cold War Two,Christopher King argues that the US and NATO a...
3,2008-08-13,0,Russian forces sink Georgian ships,of CNN readers Russia s actions in Georgia jus...,US warns against Israeli plan to strike agains...,Russian forces sink Georgian ships of CNN read...
4,2008-08-14,1,War in South Osetia pictures made by a Russian...,Russia exaggerated the death toll in South Oss...,The US and Poland have agreed a preliminary de...,War in South Osetia pictures made by a Russian...
...,...,...,...,...,...,...
1586,2014-11-25,0,Anti Slavery Campaigners Arrested in Mauritani...,Russia says it loses billions to oil slump san...,Islamic State Australian initiated air strike ...,Anti Slavery Campaigners Arrested in Mauritani...
1587,2014-11-26,1,India to raise smoking age to,Rare Shakespeare volume discovered in French l...,French President s office formally announces i...,India to raise smoking age to Rare Shakespeare...
1588,2014-11-28,1,Venezuela s representative storm out of meetin...,State of emergency in Ukraine as Russia cuts o...,Mexicos President Announces Federal Takeover o...,Venezuela s representative storm out of meetin...
1589,2014-12-01,0,ISIS may possesss nuclear material stolen from...,Russia to give Iran grain in return for oil,Netanyahu s government on verge of collapse ov...,ISIS may possesss nuclear material stolen from...


In [12]:
columns = ['Date', '0', '1', '2']
df_train = df_train.drop(columns, axis=1)
df_valid = df_valid.drop(columns, axis=1)
df_test = df_test.drop(columns, axis=1)

In [13]:
df_train.head()

,Label,new_combined_news
0,0,Georgia downs two Russian warplanes as countri...
1,1,War in South Ossetia Israel and the US behind ...
2,0,Christopher King argues that the US and NATO a...
3,0,Russian forces sink Georgian ships of CNN read...
4,1,War in South Osetia pictures made by a Russian...


In [14]:
max_len = 0
for i in range(df_train.shape[0]):
  leng = len(df_train.iloc[i, -1].split())
  if max_len < leng:
    max_len = leng
print(max_len)

129


In [ ]:
def stopwords_removal(news):
  word_list = news.split()
  tmp = []
  for word in word_list:
    if word.lower() not in stopwords.words('english'):
      tmp.append(word)
    filtered_news_article = ' '.join(tmp)
    return filtered_news_article

In [15]:
#train_data = (df_train['combined_news'].to_list(), df_train['Label'].to_list()) 
train_data = (df_train['new_combined_news'].to_list(), df_train['Label'].to_list()) 

In [16]:
#test_data = (df_test['combined_news'].to_list(), df_test['Label'].to_list())
test_data = (df_test['new_combined_news'].to_list(), df_test['Label'].to_list())

In [17]:
def create_dataset(texts: List[str],labels: List[int],num_classes: int = 2,class_names: List[int] = [0, 1]) -> Dataset:
    """
    Create HuggingFace Dataset from texts and labels.
    """
    ds = Dataset.from_dict({'text': texts, 'label': labels})
    ds = ds.cast_column('label', ClassLabel(num_classes=num_classes, names=class_names))
    return ds

In [18]:
def fine_tune(
    train_data: Tuple[List[str], List[int]], # text, label
    eval_data: Tuple[List[str], List[int]],
    model_output_dir: Union[str, Path],
    model_name: str = "distilbert-base-uncased",
    num_classes: int = 2,
    class_names: List[int] = [0, 1],
    learning_rate: float = 1e-5,
    weight_decay: float = 0.3,
    steps: int = 50,
    batch_size: int = 16,
    epochs: int = 20,
    verbose: bool = True
) -> Trainer:
    """
    Fine-tune a transformer for text classification.
    """

    def model_init():
        return AutoModelForSequenceClassification.from_pretrained(
            model_name, num_labels=num_classes
        )
    
    # create datasets
    if verbose:
        print('Creating train and evaluation datasets.')
    train_ds = create_dataset(train_data[0], train_data[1], num_classes, class_names)
    eval_ds = create_dataset(eval_data[0], eval_data[1], num_classes, class_names)

    # perform shuffling
    train_ds = train_ds.shuffle(seed=42)
    eval_ds = eval_ds.shuffle(seed=42)

    if verbose:
        print(f'Tokenizing inputs.')
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    tokenize = lambda eg: tokenizer(eg['text'], truncation=True)
    tokenized_train = train_ds.map(tokenize, batched=True)
    tokenized_eval = eval_ds.map(tokenize, batched=True)
    data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

    # define training arguments
    training_args = TrainingArguments(
        output_dir=model_output_dir,
        evaluation_strategy='steps',#steps or epoch
        # save_strategy = 'epoch',
        eval_steps=steps,
        logging_strategy='steps',
        logging_steps=steps,
        #saving_strategy='steps',
        #saving_steps=steps,
        save_strategy='steps',
        save_steps=steps,
        learning_rate=learning_rate,
        weight_decay=weight_decay,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        num_train_epochs=epochs, 
        load_best_model_at_end=True,
    )

    # perform fine-tuning
    if verbose:
        print("Fine-tuning model.")
    trainer = Trainer(
        model_init=model_init, 
        args=training_args,
        train_dataset=tokenized_train,
        eval_dataset=tokenized_eval,
        tokenizer=tokenizer,
        data_collator=data_collator,
        callbacks=[EarlyStoppingCallback(early_stopping_patience=5, early_stopping_threshold=0.0)]
    )
    trainer.train()
    # save model
    if verbose:
        print(f'Saving best model to {model_output_dir}/best-model.')
    trainer.save_model(f'{model_output_dir}/best-model')
    return trainer


In [19]:
def predict(trainer: Trainer, texts: List[str]) -> Tuple[List[int], List[float]]:
    """
    Return predicted class label and probabilities.
    """

    # create dataset and tokenize
    ds = Dataset.from_dict({'text': texts})
    tokenize = lambda eg: trainer.tokenizer(eg['text'], truncation=True, padding=True)
    tokenized_ds = ds.map(tokenize, batched=True)
    # make predictions
    logits = trainer.predict(tokenized_ds).predictions
    softmax = scipy.special.softmax(logits, axis=-1)
    prediction = softmax.argmax(axis=-1).tolist()
    probabilities = softmax.tolist()
    return prediction, probabilities

In [20]:
def evaluate(y: List[int], y_pred: List[int], class_names: List[int] = [0, 1], verbose=True):
    """
    Return evaluation metrics and print the confusion matrix.
    """

    # generate confusion matrix
    cf_mat = metrics.confusion_matrix(y, y_pred)
    row_names = ['Actual ' + str(c) for c in class_names]
    col_names = ['Predicted ' + str(c) for c in class_names]
    cf_mat = pd.DataFrame(cf_mat, index=row_names, columns=col_names)

    # calculate performance metrics
    accuracy = metrics.accuracy_score(y, y_pred)
    precision = metrics.precision_score(y, y_pred, average=None)
    recall = metrics.recall_score(y, y_pred, average=None)
    f1 = metrics.f1_score(y, y_pred, average=None)

    output_dict = {}
    output_dict['Accuracy'] = accuracy
    output_dict['Precision'] = precision
    output_dict['Recall'] = recall
    output_dict['F1'] = f1

    #print("Debug1:") ###
    if verbose:
        print(cf_mat)
        for k, v in output_dict.items():
            #print("Debug2:") ###
            print(k, ': ', v)
    
    return output_dict

In [21]:
def fine_tune_pipeline(
    train_data: Tuple[List[str], List[int]], # text, label
    eval_data: Tuple[List[str], List[int]],
    model_output_dir: Union[str, Path],
    model_name: str = "distilbert-base-uncased",
    num_classes: int = 2,
    class_names: List[int] = [0, 1],
    learning_rate: float = 1e-5,
    weight_decay: float = 0.3,
    steps: int = 50,
    batch_size: int = 16,
    epochs: int = 20,
    verbose: bool = True
) -> Dict[str, Any]:
    """
    Pipeline for fine-tuning a transformer and evaluating the performance.
    """

    # fine-tuning
    trainer = fine_tune(
        train_data,
        eval_data,
        model_output_dir,
        model_name,
        num_classes,
        class_names,
        learning_rate,
        weight_decay,
        steps,
        batch_size,
        epochs,
        verbose
    )

    # predict and evaluate on train & validation data
    train_pred, train_proba = predict(trainer, train_data[0])
    eval_pred, eval_proba = predict(trainer, eval_data[0])

    if verbose:
        print('Performance of tuned model on training set:')
    train_metrics = evaluate(train_data[1], train_pred, class_names)
    print()
    if verbose:
        print('Performance of tuned model on evaluation set:')
    eval_metrics = evaluate(eval_data[1], eval_pred, class_names)

    output = {
        'train_prediction': train_pred,
        'train_probabilities': train_proba,
        'train_performance': train_metrics,
        'eval_prediction': eval_pred,
        'eval_probabilities': eval_proba,
        'eval_performance': eval_metrics,
    }

    return output

In [22]:
save_model_dir = "/content/drive/MyDrive/5246Project/save_weights"
output = fine_tune_pipeline(train_data=train_data, eval_data=test_data, model_output_dir=save_model_dir, learning_rate=5e-6)

Creating train and evaluation datasets.


Casting the dataset:   0%|          | 0/1591 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/199 [00:00<?, ? examples/s]

Tokenizing inputs.


Map:   0%|          | 0/1591 [00:00<?, ? examples/s]

...

Saving best model to /content/drive/MyDrive/5246Project/save_weights/best-model.


Map:   0%|          | 0/1591 [00:00<?, ? examples/s]

Map:   0%|          | 0/199 [00:00<?, ? examples/s]

Performance of tuned model on training set:
          Predicted 0  Predicted 1
Actual 0            0          728
Actual 1            0          863
Accuracy :  0.5424261470773099
Precision :  [0.         0.54242615]
Recall :  [0. 1.]
F1 :  [0.         0.70334148]

Performance of tuned model on evaluation set:
          Predicted 0  Predicted 1
Actual 0            0           92
Actual 1            0          107
Accuracy :  0.5376884422110553
Precision :  [0.         0.53768844]
Recall :  [0. 1.]
F1 :  [0.         0.69934641]


/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
